# Setting up an accessibility analysis Mozambique : Downloading the Road network

In [ ]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import shape

In [ ]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper,)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter


In [ ]:
root_dir = Path(r'P:\moonshot2-casestudy\RA2CE_HACKATHON_JULY\Automating_Destinations_and_buffer_creation')
Extent = root_dir/"Buffer_Polygons"/"buffer_polygon_network.geojson"
Output_Destinations = root_dir.joinpath('static','network')
assert root_dir.exists()

In [ ]:
gdf = gpd.read_file(Extent)
gdf.explore()

In [ ]:
network_polygon = shape(gdf.geometry.iloc[0])
print(network_polygon)

In [ ]:
#First we define which roads we will want to download from OSM to create a network with
_network_section = NetworkSection(
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[RoadTypeEnum.MOTORWAY,RoadTypeEnum.MOTORWAY_LINK,RoadTypeEnum.PRIMARY, RoadTypeEnum.PRIMARY_LINK,RoadTypeEnum.TRUNK, RoadTypeEnum.SECONDARY,RoadTypeEnum.SECONDARY_LINK, RoadTypeEnum.TERTIARY, RoadTypeEnum.TERTIARY_LINK], 
)

_network_config_data = NetworkConfigData(
    root_path= root_dir,
    output_path= root_dir/"output",
    network=_network_section,
    )

_network_config_data = NetworkConfigData(network=_network_section, static_path=root_dir.joinpath('static'))
_graph,_gdf = OsmNetworkWrapper.get_network_from_polygon(_network_config_data, network_polygon)

In [ ]:
# Export the graph
_exporter = MultiGraphNetworkExporter(basename='base_graph', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=root_dir.joinpath('static','output_graph'), export_data=_graph)

# Export the network
_exporter = GeoDataFrameNetworkExporter(basename='base_network', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=root_dir.joinpath('static','output_graph'), export_data=_gdf)